In [32]:
import os
import cv2
import numpy as np
from keras import layers
from keras.layers import Input,Add,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D,AveragePooling2D,MaxPooling2D
from keras.models import Model,load_model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import tensorflow as tf
from tensorflow.keras import backend as k

In [8]:
def identity_block(X,f,filters,stage,block):
    #defining name basis
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'
    #Retrieve filters
    F1,F2,F3=filters
    #save the input value. we will need this later to add back to main path
    X_shortcut=X
    #First component of main path
    X=Conv2D(filters=F1,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X=Activation('relu')(X)
    
    #second component of main path
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X=Activation('relu')(X)
    #third component of main path
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    
    # Final step : Add shortcut value to main path and pass it through RELU activation
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    return X
    

In [13]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
#tf.reset_default_graph()
with tf.compat.v1.Session() as test:
    A_prev=tf.compat.v1.placeholder("float",[3,4,4,6])
    X=np.random.randn(3,4,4,6)
    A=identity_block(A_prev,f=2,filters=[2,4,6],stage=1,block='a')
    test.run(tf.compat.v1.global_variables_initializer())
    out=test.run([A],feed_dict={A_prev:X,k.learning_phase():0})
    print("out= ",out[0][1][1][0])

out=  [0.        2.1321862 1.2955861 0.        4.0805316 1.2945693]


In [22]:
def convolutional_block(X,f,filters,stage,block,s=2):
    #defining name basis
    conv_name_base='res'+str(stage)+block+'_branch'
    bn_name_base='bn'+str(stage)+block+'_branch'
    #Retrieve filters
    F1,F2,F3=filters
    #save the input value. we will need this later to add back to main path
    X_shortcut=X
    #First component of main path
    X=Conv2D(filters=F1,kernel_size= (1,1), strides=(s,s),name=conv_name_base +'2a',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
    X=Activation('relu')(X)
    
    #second component of main path
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',name=conv_name_base +'2b',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
    X=Activation('relu')(X)
    #third component of main path
    X=Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)
    
    #Shortcut path
    X_shortcut=Conv2D(F3,kernel_size=(1,1),strides=(1,1),name=conv_name_base+'1',kernel_initializer=glorot_uniform(seed=0))(X)
    X_shortcut=BatchNormalization(axis=3,name=bn_name_base+'1')(X_shortcut)
    # Final step : Add shortcut value to main path and pass it through RELU activation
    X=Add()([X,X_shortcut])
    X=Activation('relu')(X)
    return X
    

In [23]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
#tf.reset_default_graph()
with tf.compat.v1.Session() as test:
    A_prev=tf.compat.v1.placeholder("float",[3,4,4,6])
    X=np.random.randn(3,4,4,6)
    A=convolutional_block(A_prev,f=2,filters=[2,4,6],stage=1,block='a')
    test.run(tf.compat.v1.global_variables_initializer())
    out=test.run([A],feed_dict={A_prev:X,k.learning_phase():0})
    print("out= ",out[0][1][1][0])

out=  [1.0681336 2.3918095 2.7993917 1.4203122 2.046123  1.9735686]


In [24]:
def ResNet50(input_shape=(64,64,3),classes=2):
    #Define the input as a tensor with shape input_shape
    X_input=Input(input_shape)
    #Zeror_padding
    X=ZeroPadding2D((3,3))(X_input)
    #stage 1
    X=Conv2D(64,(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3,name='bn_conv1')(X)
    X=Activation('relu')(X)
    X=MaxPooling2D((3,3),strides=(2,2))(X)
    #stage 2
    X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
    X=identity_block(X,3,[64,64,256],stage=2,block='b')
    X=identity_block(X,3,[64,64,256],stage=2,block='c')
    #stage 3
    X=convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
    X=identity_block(X,3,[128,128,512],stage=3,block='b')
    X=identity_block(X,3,[128,128,512],stage=3,block='c')
    X=identity_block(X,3,[128,128,512],stage=3,block='d')
    
    #stage 4
    X=convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
    X=identity_block(X,3,[256,256,1024],stage=4,block='b')
    X=identity_block(X,3,[256,256,1024],stage=4,block='c')
    X=identity_block(X,3,[256,256,1024],stage=4,block='d')
    X=identity_block(X,3,[256,256,1024],stage=4,block='e')
    X=identity_block(X,3,[256,256,1024],stage=4,block='f')
    
    #stage 5
    X=convolutional_block(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
    X=identity_block(X,3,[512,512,2048],stage=5,block='b')
    X=identity_block(X,3,[512,512,2048],stage=5,block='c')
    
    #AVGpool
    X=AveragePooling2D((2,2),name='avg_pool')(X)
    
    #outer Layer
    X=Flatten()(X)
    X=Dense(classes,activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0))(X)
    
    #create model
    model=Model(inputs=X_input,outputs=X,name='ReSNet50')
    return model
    
    
    
    

In [29]:
ResNet50()

In [26]:
    ROWS=64
    COLS=64
    CHANNEL=3
    CLASSES=2

In [27]:
model=ResNet50(input_shape=(ROWS,COLS,CHANNEL),classes=CLASSES)

In [28]:
print(model.summary())

Model: "ReSNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 70, 70, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [33]:
plot_model(model,to_file='model.png')
SVG(model_to_dot(model).create(prog='dot',format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.